In [8]:
# Python 2/3 compatibility
from __future__ import print_function

import numpy as np
import cv2
import video
import imgUtils
from common import anorm2, draw_str

In [18]:
#Params for the Shi-Tomasi corner detector
#maxCorners – Maximum number of corners to return. If there are more corners than are found, 
    #the strongest of them is returned.
#qualityLevel – Parameter characterizing the minimal accepted quality of image corners. 
    #The parameter value is multiplied by the best corner quality measure, 
    #which is the minimal eigenvalue. The corners with the quality measure less than the product are rejected. 
    #For example, if the best corner has the quality measure = 1500, and the qualityLevel=0.01 , 
    #then all the corners with the quality measure less than 15 are rejected.
#minDistance – Minimum possible Euclidean distance between the returned corners.
#blockSize – Size of an average block for computing a derivative covariation matrix over each pixel neighborhood. 
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.2,
                       minDistance = 2,
                       blockSize = 2 )

#Params for the LK optical flow
#winSize – size of the search window at each pyramid level.
#maxLevel – 0-based maximal pyramid level number; if set to 0, 
    #pyramids are not used (single level), if set to 1, two levels are used, and so on; 
    #if pyramids are passed to input then algorithm will use as many levels as pyramids 
    #have but no more than maxLevel.
#criteria – parameter, specifying the termination criteria of the iterative search algorithm:
    #After the specified maximum number of iterations criteria.maxCount or when the search window moves 
    #by less than criteria.epsilon.
lk_params = dict( winSize  = (5, 5),
                  maxLevel = 3,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))



In [23]:
rows = 250
cols = 250

rec_sizex = 8
rec_sizey = 50
rec_posx = 1
rec_posy = 100
mov_size = 5


# Create some random colors
color = np.random.randint(0,255,(20,1))

#create a empty img and draw a rectangle 
gray = np.zeros((rows,cols),np.uint8)
cv2.rectangle(gray,(rec_posx,rec_posy),(rec_posx+rec_sizex,rec_posy+rec_sizey),255,-1)
rec_posx+= mov_size

#find corners in the created frame
p0 = cv2.goodFeaturesToTrack(gray, mask = None, **feature_params) 

#create a mask for drawing tracks later
mask = np.zeros_like(gray)

cv2.startWindowThread()
cv2.namedWindow("img")


while (rec_posx < cols-rec_sizex):
    
    if p0 is None :
        print("p0 is empty (calculating new one) \n P0_old:")
        print(p0)
        p0 = cv2.goodFeaturesToTrack(gray, mask = None, **feature_params) 
        print("P0_new:")
        print(p0)
    
    #move the rectangle by 'mov_size" pixels to the left
    prevgray = gray
    gray = np.zeros((rows,cols),np.uint8)
    cv2.rectangle(gray,(rec_posx,rec_posy),(rec_posx+rec_sizex,rec_posy+rec_sizey),255,-1)
    rec_posx += mov_size
    
    #calc sparse optical flow
    p1, st, err = cv2.calcOpticalFlowPyrLK(prevgray, gray, p0, None, **lk_params)
    
    if p1 is not None :
        #select good points
        good_new = p1[st==1]
        good_old = p0[st==1]

        #draw the movement tracks
        # draw the tracks
        for i,(new,old) in enumerate(zip(good_new,good_old)):
            a,b = new.ravel()
            c,d = old.ravel()
            mask = cv2.line(mask, (a,b),(c,d), color[i].tolist(), 2)
            prevgray = cv2.circle(prevgray,(a,b),5,color[i].tolist(),-1)
        img = cv2.add(prevgray,mask)

        #update the new points to track
        p0 = good_new.reshape(-1,1,2)


        #prints the img
        cv2.imshow('img',img)
        ch = cv2.waitKey(0)
        if ch == 27:
            break
    else:
        p0 = p1
        print("No features were tracked by LK algorithm")
cv2.destroyAllWindows()